In [1]:
#Import earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=pcgqHFMWr9wrcOpiPHffclp3CQwLD8pWk0gOpQs_ync&tc=QQ7Mx0FndQFh2ZpzUjM2506pz0pt68isntWlHsdICZg&cc=3rALWghCVCUGfYeHmxNlrTNSjgzReM_OPApgGQmdXCA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhVSQFME8ZzIjDuPANuzVDJJm5D99O2WvGskA8AePsQn8xdEas4nnS8

Successfully saved authorization token.


In [2]:
#Get a feature collection with Egypt
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Turkey = countries.filter(ee.Filter.eq('country_na', 'Turkey'))

In [3]:
# Load a pre-computed Landsat composite for input
input = ee.Image('LANDSAT/LE7_TOA_1YEAR/2001')

#Define a region in which to generate a sample of the input
region = ee.Geometry.Rectangle(29.7, 30, 32.5, 31.7)

# Make the training dataset
training = input.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000
})

In [4]:
#Initiate the clusterer and train it
clusterer = ee.Clusterer.wekaKMeans(15).train(training)

In [5]:
# Cluster the input using the training clustrer
result = input.cluster(clusterer)

In [6]:
# Clip the model result
clip = result.clip(Turkey)

In [7]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[25, 29], zoom_start = 6)

# Add the layer to the map object.
my_map.add_ee_layer(clip.randomVisualizer(), {}, 'clusters')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)